<a href="https://colab.research.google.com/github/crossmodaldebate/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os

def mount_google_drive():
    drive.mount('/content/drive')

def ensure_directory_exists_on_drive(relative_path):
    drive_path = os.path.join('/content/drive/MyDrive', relative_path)
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)
    return drive_path

def save_fig(fig, filename, drive_folder_path):
    filepath = os.path.join(drive_folder_path, filename)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close(fig)

def save_summary(summary_text, drive_folder_path):
    filepath = os.path.join(drive_folder_path, "sumario.txt")
    with open(filepath, "w") as f:
        f.write(summary_text)

def plot_violin(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.violinplot(x=x_col, y=y_col, data=df, hue=hue, palette="viridis", split=(hue is not None), ax=ax)
    ax.set_title(title, color='cyan')
    ax.set_xlabel(x_col.replace('_', ' '), color='white')
    ax.set_ylabel(y_col.replace('_', ' '), color='white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.set_facecolor('black')
    fig.set_facecolor('black')
    if hue:
        ax.legend(title=hue.replace('_', ' '), facecolor='black', edgecolor='white', labelcolor='white')
    save_fig(fig, filename, drive_folder_path)

def plot_kde(df, col, title, filename, drive_folder_path, hue=None):
    fig, ax = plt.subplots(figsize=(12, 8))
    if hue:
        for level in df[hue].unique():
            sns.kdeplot(df.loc[df[hue] == level, col], label=level, fill=True, ax=ax, color='cyan')
        ax.legend(title=hue.replace('_', ' '), facecolor='black', edgecolor='white', labelcolor='white')
    else:
        sns.kdeplot(df[col], fill=True, ax=ax, color='cyan')
    ax.set_title(title, color='cyan')
    ax.set_xlabel(col.replace('_', ' '), color='white')
    ax.set_ylabel('Densidade', color='white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.set_facecolor('black')
    fig.set_facecolor('black')
    save_fig(fig, filename, drive_folder_path)

def plot_line(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.lineplot(x=x_col, y=y_col, data=df, hue=hue, palette="viridis", ax=ax)
    ax.set_title(title, color='cyan')
    ax.set_xlabel(x_col.replace('_', ' '), color='white')
    ax.set_ylabel(y_col.replace('_', ' '), color='white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.set_facecolor('black')
    fig.set_facecolor('black')
    if hue:
        ax.legend(title=hue.replace('_', ' '), facecolor='black', edgecolor='white', labelcolor='white')
    save_fig(fig, filename, drive_folder_path)

def plot_stacked_bar(df, x_col, y_cols, title, filename, drive_folder_path):
    fig, ax = plt.subplots(figsize=(12, 8))
    df.plot(kind='bar', stacked=True, ax=ax, color=['cyan', 'magenta', 'yellow'])
    ax.set_title(title, color='cyan')
    ax.set_xlabel(x_col.replace('_', ' '), color='white')
    ax.set_ylabel('Valores', color='white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.set_facecolor('black')
    fig.set_facecolor('black')
    save_fig(fig, filename, drive_folder_path)

def analise_exploratoria(dados, drive_folder_path):
    descritivas = dados.describe(include='all')
    save_summary(descritivas.to_string(), drive_folder_path)
    plot_kde(dados, 'desempenho_prova', 'Distribuição do Desempenho na Prova', 'desempenho_prova_distribuicao.png', drive_folder_path)

def analisar_impacto_descentralizacao(dados, drive_folder_path):
    dados['aluno_id'] = dados['aluno_id'].astype('category')
    plot_violin(dados, 'grupo', 'desempenho_prova', 'Desempenho por Grupo (Violin)', 'desempenho_grupo_violin.png', drive_folder_path, hue='grupo')
    plot_line(dados, 'satisfacao', 'desempenho_prova', 'Desempenho ao Longo da Satisfação', 'desempenho_satisfacao.png', drive_folder_path, hue='grupo')

def analisar_series_temporais_ideb(dados, drive_folder_path):
    plot_line(dados, 'diversidade_equipe', 'desempenho_prova', 'Desempenho Médio ao Longo da Diversidade da Equipe', 'desempenho_diversidade.png', drive_folder_path)

def analise_xgboost(dados, drive_folder_path):
    pass

def analisar_participacao_atividades(dados, drive_folder_path):
    plot_line(dados, 'participacao_atividades', 'desempenho_prova', 'Desempenho ao Longo da Participação em Atividades', 'desempenho_participacao.png', drive_folder_path, hue='grupo')

def analisar_frequencia(dados, drive_folder_path):
    plot_line(dados, 'frequencia', 'desempenho_prova', 'Desempenho ao Longo da Frequência', 'desempenho_frequencia.png', drive_folder_path, hue='grupo')

def analisar_centralidade_comunicacao(dados, drive_folder_path):
    plot_line(dados, 'centralidade_comunicacao', 'desempenho_prova', 'Desempenho ao Longo da Centralidade de Comunicação', 'desempenho_centralidade.png', drive_folder_path, hue='grupo')

def analisar_questionario_socioemocional(dados, drive_folder_path):
    plot_violin(dados, 'questionario_socioemocional', 'desempenho_prova', 'Desempenho por Questionário Sociemocional (Violin)', 'desempenho_questionario_violin.png', drive_folder_path, hue='grupo')

def analisar_feedback_professor(dados, drive_folder_path):
    plot_violin(dados, 'feedback_professor', 'desempenho_prova', 'Desempenho por Feedback do Professor (Violin)', 'desempenho_feedback_violin.png', drive_folder_path, hue='grupo')

def analisar_stacked_bar(dados, drive_folder_path):
    stacked_data = dados.groupby('grupo')[['satisfacao', 'centralidade_comunicacao', 'diversidade_equipe']].mean()
    plot_stacked_bar(stacked_data, 'grupo', ['satisfacao', 'centralidade_comunicacao', 'diversidade_equipe'],
                     'Médias Empilhadas por Grupo', 'medias_empilhadas_grupo.png', drive_folder_path)

if __name__ == "__main__":
    mount_google_drive()
    graficos_drive_path = ensure_directory_exists_on_drive('data')
    data_url = "https://docs.google.com/spreadsheets/d/1gIvf74i3kbPV4eEaTqky51Ee_YDL92GipfgYlxHKpwM/export?format=csv"
    dados = pd.read_csv(data_url)
    dados.columns = [col.strip().lower().replace(" ", "_") for col in dados.columns]
    dados['aluno_id'] = dados['aluno_id'].astype('category')
    analise_exploratoria(dados, graficos_drive_path)
    analisar_impacto_descentralizacao(dados, graficos_drive_path)
    analisar_series_temporais_ideb(dados, graficos_drive_path)
    analise_xgboost(dados, graficos_drive_path)
    analisar_participacao_atividades(dados, graficos_drive_path)
    analisar_frequencia(dados, graficos_drive_path)
    analisar_centralidade_comunicacao(dados, graficos_drive_path)
    analisar_questionario_socioemocional(dados, graficos_drive_path)
    analisar_feedback_professor(dados, graficos_drive_path)
    analisar_stacked_bar(dados, graficos_drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-c024e96ad27e>:38: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(title=hue.replace('_', ' '), facecolor='black', edgecolor='white', labelcolor='white')
<ipython-input-4-c024e96ad27e>:60: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.lineplot(x=x_col, y=y_col, data=df, hue=hue, palette="viridis", ax=ax)
